## A simple notebook to test some functions

#### Activation functions: sigmoid and ReLU

Sigmoid and its derivative

In [1]:
import numpy as np

def sigmoid(x):

    s = 1/(1 + np.exp(-x))

    return s

def derivative_sigmoid(s):

    ds = s*(1-s)

    return ds

w = [1,2]
x = [2,4]
b = 3
z = np.dot(w,x) + b
a = sigmoid(z)
da = derivative_sigmoid(a)
print(a,da)


0.999997739675702 2.260319188887599e-06


ReLU and its derivative

In [2]:
def relu(x):
    return x * (x > 0)

def derivative_relu(x):
    return 1 * (x>0)

Now let's implement the softmax function

In [3]:
def softmax(vector):
    e = np.exp(vector)
    return e / e.sum()

In [ ]:
def derivative_softmax(vector):
    pass

In [ ]:
def inverted_dropout(x, p):
    # p = dropout probability
    mask = (np.random.rand(*x.shape) > p).astype(float)
    x_dropped = (x * mask)/(1 - p) # the actual dropout
    return x_dropped


### Implement a fully parametrizable neural network class

In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['species'] = iris.target
data


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2,random_state=42)

X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
65,6.7,3.1,4.4,1.4
11,4.8,3.4,1.6,0.2
42,4.4,3.2,1.3,0.2
...,...,...,...,...
71,6.1,2.8,4.0,1.3
106,4.9,2.5,4.5,1.7
14,5.8,4.0,1.2,0.2
92,5.8,2.6,4.0,1.2


## Create a NN class

In [ ]:
from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2,random_state=42)

# just 4 samples
X = np.array(X_train)

# target values 
y = np.array(y_train).T 

class NeuralNetwork:

    #hidden layer (sigmoid, relu)
    #output layer (softmax)

    def __init__(self, activation_function, no_of_input_nodes, no_of_hidden_nodes, no_of_output_nodes, n_epochs,lambda1,lambda2):
        self.hidden_layers = len(no_of_hidden_nodes)
        self.activation_function = activation_function
        self.no_of_input_nodes = no_of_input_nodes # as many as the dataset's features
        self.no_of_hidden_nodes = no_of_hidden_nodes # no fixed number, needs tuning
        self.no_of_output_nodes = no_of_output_nodes # as many as the output classes
        self.n_epochs = n_epochs
        self.lambda1 = lambda1 #lambda variable for L1 regularisation
        self.lambda2 = lambda2 #lambda variable for L2 regularisation

        self.z_values = [] # need to store for backprop
        self.a_values = []   # need to store for backprop, first value is the actual data

        self.weights, self.biases = self.weights_and_bias()

    def weights_and_bias(self):
        layers = [self.no_of_input_nodes] + self.no_of_hidden_nodes + [self.no_of_output_nodes] #e.g. [2,3,5,2]
        weights = [] #TODO: weight and bias proper initialization (xavier)
        biases = []
        for i in range(len((layers))-1):

            n_in = layers[i]
            n_out = layers[i+1]

            weights.append(2*np.random.random((n_in,n_out)) - 1)
            biases.append(np.zeros((1, n_out)))
        return weights, biases

    
    def forward_pass(self, X):

        #first hidden layer needs to have the actual data
        #all other hidden layers take the result from the previous layer
        a = X
        self.a_values.append(a) # save for backprop
        
        for layer in range(self.hidden_layers):

            W = self.weights[layer]
            b = self.biases[layer]
            z = np.dot(a, W) + b
            self.z_values.append(z) # save for backprop

            if self.activation_function =='sigmoid':
                a = sigmoid(z)
            elif self.activation_function =='relu':
                a = relu(z)
            print(f'Shape of layer: {a.shape}')

            #implementing inverted dropout
            a = inverted_dropout(a, 0.5) #TODO: if testing, we don't do dropout
            #TODO: do we add the same p for all neurons and layers?

            self.a_values.append(a) # save for backprop

        #---Output Layer---
        W = self.weights[-1]
        b = self.biases[-1]
        z = np.dot(a, W) + b
        self.z_values.append(z) # save for backprop
        a = softmax(z)
        self.a_values.append(a) # save for backprop
        print(f'Shape of output layer: {a.shape}')
        return a
        

    def backward_pass():
        pass

    def compute_loss(self, y_pred, y_true):
        '''
        Cross entropy loss for multi-class classification with L1 and L2 regularization
        '''

        # number of samples
        N = y_true.shape[0]
        
        correct_probs = y_pred[np.arange(N), y_true] 
    
        # loss = average of -log(p) whre p is the predicted probability of the correct class
        loss = -np.sum(np.log(correct_probs)) / N

        l1_loss = self.lambda1* np.sum(np.abs(self.weights)) #TODO: needs to be added in weight update in backprop too
        l2_loss = self.lambda2* np.sum(np.abs(self.weights**2))

        return loss + l1_loss + l2_loss




In [ ]:
#TODO: run the forward pass in epochs
#n_epochs = 500
#for epoch in range(n_epochs):


#Param tuning:
#lambda1 and lambda 2 can be from 0 to 1
nn = NeuralNetwork('sigmoid',4,[3,4],3,500,0.2,0.3)

# Forward pass
y_pred = nn.forward_pass(X_train)

# Loss function
loss = nn.compute_loss(y_pred, y_train)

# Backward pass
nn.backward_pass(X_train, y_train, y_pred)

Shape of layer: (120, 3)
Shape of layer: (120, 4)
Shape of output layer: (120, 3)


In [10]:
print(np.min(y_pred), np.max(y_pred))
print(np.sum(y_pred, axis=1)[:5])


0.0010344313351809925 0.0051952548612967265
[0.00841486 0.00836609 0.00831748 0.00839032 0.0084097 ]


### Backward Pass